In [ ]:
!pip install --upgrade google-generativeai
!pip show google-generativeai

Name: google-generativeai
Version: 0.8.5
Summary: Google Generative AI High level API client library and tools.
Home-page: https://github.com/google/generative-ai-python
Author: Google LLC
Author-email: googleapis-packages@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: google-ai-generativelanguage, google-api-core, google-api-python-client, google-auth, protobuf, pydantic, tqdm, typing-extensions
Required-by: 


In [ ]:
import os
from google.colab import userdata
import google.generativeai as genai # Importação correta

print(f"Versão da biblioteca google-generativeai: {genai.__version__}") # Verifica a versão

try:
    # Certifique-se de que a chave 'GOOGLE_API_KEY' existe no UserData do Colab
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        raise ValueError("API Key 'GOOGLE_API_KEY' não encontrada no Colab Userdata.")

    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key configurada.")

    # Use um nome de modelo sabidamente válido
    model = genai.GenerativeModel("gemini-1.5-flash-latest")
    print("Modelo Gemini instanciado com sucesso!")

    # Opcional: Teste rápido de geração
    # response = model.generate_content("Olá!")
    # print(f"Resposta do modelo: {response.text}")

except AttributeError as ae:
    print(f"Erro de Atributo: {ae}")
    print("Verifique se a biblioteca 'google-generativeai' está instalada corretamente e se o ambiente de execução foi reiniciado após a instalação/atualização.")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")

Versão da biblioteca google-generativeai: 0.8.5
API Key configurada.
Modelo Gemini instanciado com sucesso!


In [ ]:
import google.generativeai as genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
MODEL_ID = "gemini-2.0-flash"

In [ ]:
from IPython.display import HTML, Markdown

In [ ]:
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
def obter_resposta_do_modelo(prompt):
    """Envia um prompt para o modelo Gemini e retorna a resposta."""
    if model is None: # Verifica se o modelo foi inicializado
        return "Erro: O modelo Gemini não foi inicializado corretamente."
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Ocorreu um erro ao obter a resposta do modelo: {e}"

In [ ]:
def chatbot_busca_imovel():
    print("Olá! Bem-vindo ao nosso sistema inteligente de busca de imóveis. Para começarmos, por favor, me diga o que você procura:")

    tipo_imovel = input("Qual tipo de imóvel você procura (casa, apartamento, estúdio, comercial)? ").lower()
    cidade = input(f"Em qual cidade você gostaria de buscar um(a) {tipo_imovel}? ").lower()
    dormitorios = input("Quantos dormitórios você precisa? (ex: 1, 2, 3+) ").lower()
    vaga = input("Precisa de vaga de garagem? (sim/não) ").lower()
    varanda = input("Precisa de varanda? (sim/não) ").lower()
    preco_disposto = input("Qual o valor máximo que você está disposto a pagar pelo imóvel? (ex: até 300000, entre 300000 e 500000, acima de 500000) ").lower()

    prompt_busca = f"""Você é um agente de busca de imóveis especializado em encontrar as melhores opções para o cliente.
    O cliente está buscando um imóvel do tipo: {tipo_imovel} na cidade de {cidade}, com {dormitorios} dormitórios,
    opção de vaga de garagem: {vaga}, com varanda: {varanda}, e com um preço máximo de {preco_disposto}.

    Realize uma busca simulada e apresente de 15 a 20 opções que melhor correspondam a esses critérios.
    Para cada opção, inclua um breve título (criativo e relevante), uma descrição concisa destacando os principais atrativos e um link genérico para um portal de imóveis (formato: www.portal.com.br/id_...).
    Formate a saída de forma clara e numerada para o cliente analisar.

    Após apresentar as opções, pergunte diretamente ao cliente: 'Essas opções iniciais atendem à sua busca?'"""

    #response = não consegui achar o valor correto para declarar a variável de response imprimindo o site buscado.
    tools=[google_search]
    resposta_busca = obter_resposta_do_modelo(prompt_busca)
    print(f"\nResultados da Busca:\n{resposta_busca}")
    # Exibe as URLs nas quais ele se baseou
    print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
    print()
    display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

    satisfacao = input("Essas opções iniciais atendem à sua busca? (sim/não) ").lower()

    tentativas_refinamento = 0
    max_tentativas = 2

    while satisfacao == "não" and tentativas_refinamento < max_tentativas:
        print("\n--- Refinando a Busca ---")
        detalhes_adicionais = input("Por favor, forneça mais detalhes sobre o que você procura para que eu possa refinar a busca: ").lower()
        novo_valor_maximo = input("Qual o valor máximo atualizado? ").lower()

        prompt_refinar = f"""Você é um agente de busca de imóveis dedicado a encontrar o imóvel perfeito para o cliente.
        O cliente inicialmente buscava:
        Tipo: {tipo_imovel}
        Cidade: {cidade}
        Dormitórios: {dormitorios}
        Vaga de Garagem: {vaga}
        Varanda: {varanda}
        E estava disposto a pagar até: {novo_valor_maximo}

        O cliente agora adicionou as seguintes informações para refinar a busca: {detalhes_adicionais}.

        Realize uma nova busca simulada de até 15 opções que correspondam a esses critérios atualizados.
        Para cada opção, inclua um breve título, uma descrição concisa e um link genérico para um portal de imóveis (formato: www.portal.com.br/id_...).
        Formate a saída de forma clara e numerada e pergunte ao cliente: 'Esta nova busca lhe agradou?'"""

        resposta_refinar = obter_resposta_do_modelo(prompt_refinar)
        print(f"\nResultados da Nova Busca:\n{resposta_refinar}")
        satisfacao = input("Esta nova busca lhe agradou? (sim/não) ").lower()
        tentativas_refinamento += 1
        if satisfacao == "sim":
            break
        elif tentativas_refinamento < max_tentativas:
            print("Agente: Entendo. Vamos tentar refinar mais uma vez.")
        else:
            print("Agente: Lamento não encontrar opções ideais no momento. Sugiro falar com um de nossos corretores para assistência personalizada.")
            break

    # --- Oferecer Corretor ---
    if satisfacao == "sim":
        link_formulario = "https://docs.google.com/forms/d/e/1FAIpQLSdE0lPpyja0SsRhmgSuNLfBwiCX2w7Rf8ED_q-j1fpceUNJ9g/viewform?usp=header" # Substitua pelo seu link
        deseja_corretor = input("Agente: Ótimo! Deseja o contato de um de nossos corretores para agendar uma visita ou obter mais informações? (sim/não) ").lower()
        if deseja_corretor == "sim":
            print(f"Agente: Por favor, preencha este formulário para solicitar o contato de um corretor: {link_formulario}")
        else:
            print("Agente: Agradecemos o seu contato! Se tiver mais alguma dúvida, pode perguntar.")
    if satisfacao == "nao":
      link_formulario = "https://docs.google.com/forms/d/e/1FAIpQLSdE0lPpyja0SsRhmgSuNLfBwiCX2w7Rf8ED_q-j1fpceUNJ9g/viewform?usp=header" # Substitua pelo seu link
      deseja_corretor = input("Agente: Lamento não encontrar opções ideais no momento. Deseja o contato de um de nossos corretores para agendar uma visita ou obter mais informações? (sim/não) ").lower()
      if deseja_corretor == "sim":
            print(f"Agente: Por favor, preencha este formulário para solicitar o contato de um corretor: {link_formulario}")
    else:
        print("Agente: Agradecemos o seu contato! Se tiver mais alguma dúvida, pode perguntar.")

    print("\nObrigado por utilizar nosso sistema! Tenha um ótimo dia.")

In [134]:
if __name__ == "__main__":
    chatbot_busca_imovel()

Olá! Bem-vindo ao nosso sistema inteligente de busca de imóveis. Para começarmos, por favor, me diga o que você procura:
Qual tipo de imóvel você procura (casa, apartamento, estúdio, comercial)? apartamento
Em qual cidade você gostaria de buscar um(a) apartamento? Itapevi
Quantos dormitórios você precisa? (ex: 1, 2, 3+) 2
Precisa de vaga de garagem? (sim/não) Sim
Precisa de varanda? (sim/não) Sim
Qual o valor máximo que você está disposto a pagar pelo imóvel? (ex: até 300000, entre 300000 e 500000, acima de 500000) 300000

Resultados da Busca:
Olá! Preparei uma seleção de apartamentos em Itapevi que se encaixam nos seus critérios de busca (2 dormitórios, vaga de garagem, varanda e até R$ 300.000).  Lembre-se que os preços e disponibilidades podem variar, então é importante verificar diretamente nos portais.


**Opções de Apartamentos em Itapevi:**

1. **"O Refúgio da Família":**  Apartamento aconchegante com 2 dormitórios, varanda gourmet com churrasqueira, vaga de garagem coberta e ar

Essas opções iniciais atendem à sua busca? (sim/não) não

--- Refinando a Busca ---
Por favor, forneça mais detalhes sobre o que você procura para que eu possa refinar a busca: preciso do link para o anúncio
Qual o valor máximo atualizado? 300000

Resultados da Nova Busca:
Olá! Realizei uma nova busca de apartamentos em Itapevi com base em seus critérios atualizados.  Abaixo estão 15 opções que se encaixam no seu perfil, incluindo links simulados para anúncios:


1. **Apartamento aconchegante com 2 dormitórios e vaga** -  Apartamento em ótimo estado, com 2 dormitórios, sala, cozinha, banheiro, varanda e 1 vaga de garagem. Próximo a comércios e transporte público.  www.portal.com.br/id_1234
2. **Apartamento moderno em condomínio fechado** -  Apartamento novo, 2 dormitórios, suíte, varanda gourmet, 1 vaga de garagem, lazer completo. www.portal.com.br/id_5678
3. **Apartamento com varanda ampla e vista livre** -  2 dormitórios, sala ampla, cozinha, banheiro, varanda com linda vista, 1 vaga